In [121]:
import os 
import json
import pandas as pd
import traceback

In [122]:
from dotenv import load_dotenv

load_dotenv()

True

In [123]:
key = os.getenv("GROQ_API_KEY")

In [124]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [128]:
llm = ChatGroq(
    model="llama3-8b-8192",
    groq_api_key=key,
    temperature=0.3,  # Lower temperature for consistent, factual responses
    max_tokens=2096,  # Higher limit for detailed MCQ explanations
    top_p=0.8,       # Slightly lower for more focused responses
    stop=["\n\nQuestion:", "\n\n---", "\n\n###"]  # Stop at question boundaries
)

llm_json_fixer = ChatGroq(
    model="llama3-70b-8192",
    groq_api_key=key,
    temperature=0,
    max_tokens=4096
)


d:\MCQ_Generator\env\lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [125]:
RESPONSE_JSON = {
  "quiz_info": {
    "title": "Quiz Title",
    "subject": "Subject Name",
    "difficulty": "easy/medium/hard",
    "total_questions": 3
  },
  "questions": [
    {
      "id": 1,
      "question": "What is the main topic discussed in the text?",
      "options": {
        "A": "Option A text",
        "B": "Option B text",
        "C": "Option C text",
        "D": "Option D text"
      },
      "correct_answer": "A",
      "explanation": "Brief explanation of why A is correct"
    },
    {
      "id": 2,
      "question": "Which of the following statements is true?",
      "options": {
        "A": "Option A text",
        "B": "Option B text",
        "C": "Option C text",
        "D": "Option D text"
      },
      "correct_answer": "B",
      "explanation": "Brief explanation of why B is correct"
    },
    {
      "id": 3,
      "question": "What would be the best approach for this scenario?",
      "options": {
        "A": "Option A text",
        "B": "Option B text",
        "C": "Option C text",
        "D": "Option D text"
      },
      "correct_answer": "C",
      "explanation": "Brief explanation of why C is correct"
    }
  ]
}

In [126]:
TEMPLATE = """
SYSTEM: You are a JSON-only MCQ generator. You must output ONLY valid JSON.

INPUT TEXT:
{text}

TASK: Create {number} MCQs for {subject} students ({tone} tone)

OUTPUT: ONLY valid JSON in this exact format:

{response_json}

CRITICAL JSON RULES:
✓ MUST start with {{ and end with }}
✓ All keys in double quotes: "key"
✓ All string values in double quotes: "value"  
✓ Separate with commas: "key": "value",
✓ No trailing commas before }} or ]
✓ Every question has: "id", "question", "options", "correct_answer", "explanation"
✓ Options object has: "A", "B", "C", "D" keys
✓ Correct answer is: "A", "B", "C", or "D"

FORBIDDEN:
✗ No text before or after JSON
✗ No markdown formatting
✗ No comments or explanations
✗ No single quotes
✗ No trailing commas
✗ Do NOT start with "quiz_info" - must start with {{

RESPOND WITH ONLY THE JSON OBJECT STARTING WITH {{ AND ENDING WITH }}.
"""

In [129]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [130]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [97]:
# TEMPLATE2 = """
# # Quiz Review and Enhancement Instructions

# ## Input Quiz
# {quiz}

# ## Task Description
# You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students, you need to evaluate the complexity of the questions and provide a complete analysis of the quiz.

# ## Review Requirements
# 1. **Complexity Analysis**: Evaluate if questions match student cognitive abilities (max 50 words)
# 2. **Quality Assessment**: Check grammar, clarity, and educational value
# 3. **Difficulty Adjustment**: Update questions that don't match student level
# 4. **Tone Optimization**: Adjust tone to perfectly fit student abilities
# 5. **Content Validation**: Ensure questions are appropriate for {subject} students

# ## Analysis Guidelines
# - **Cognitive Level**: Assess if questions are too easy, too hard, or just right
# - **Language Complexity**: Check vocabulary and sentence structure
# - **Question Clarity**: Ensure questions are unambiguous and well-written
# - **Option Quality**: Verify distractors are plausible but clearly incorrect
# - **Educational Value**: Confirm questions promote learning and understanding

# ## Enhancement Instructions
# - If questions are too complex: Simplify language and reduce difficulty
# - If questions are too simple: Increase complexity and add analytical elements
# - If tone is inappropriate: Adjust to match student age and subject level
# - If grammar is poor: Correct all grammatical and punctuation errors
# - If explanations are unclear: Improve clarity and educational value

# ## Output Format
# Provide your analysis and any necessary updates in the following structure:

# ### COMPLEXITY ANALYSIS
# [Your 50-word complexity assessment]

# ### QUALITY ASSESSMENT
# [Grammar, clarity, and educational value review]

# ### ENHANCED QUIZ
# [Updated quiz with improved questions, tone, and difficulty level]

# ### CHANGES MADE
# [List of specific improvements and modifications]

# ## Instructions
# - Be thorough but concise in your analysis
# - Focus on educational appropriateness and student engagement
# - Maintain the original quiz structure while improving quality
# - Ensure all changes enhance learning outcomes
# """

In [131]:
TEMPLATE2 = """
# Quiz Review and Analysis Instructions

## Input Quiz
{quiz}

## Task Description
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students, you need to evaluate the complexity of the questions and provide a complete analysis of the quiz.

## Review Requirements
1. **Complexity Analysis**: Evaluate if questions match student cognitive abilities (max 50 words)
2. **Quality Assessment**: Check grammar, clarity, and educational value
3. **Difficulty Evaluation**: Assess if questions are appropriate for student level
4. **Tone Analysis**: Evaluate if tone fits student abilities
5. **Content Validation**: Ensure questions are appropriate for {subject} students

## Analysis Guidelines
- **Cognitive Level**: Assess if questions are too easy, too hard, or just right
- **Language Complexity**: Check vocabulary and sentence structure
- **Question Clarity**: Evaluate if questions are unambiguous and well-written
- **Option Quality**: Verify if distractors are plausible but clearly incorrect
- **Educational Value**: Confirm if questions promote learning and understanding

## Output Format
Provide your analysis in the following structure:

### COMPLEXITY ANALYSIS
[Your 50-word complexity assessment]

### QUALITY ASSESSMENT
[Grammar, clarity, and educational value review]

### DIFFICULTY EVALUATION
[Assessment of question difficulty relative to student level]

### TONE ANALYSIS
[Evaluation of language and tone appropriateness]

### CONTENT VALIDATION
[Assessment of subject matter appropriateness and educational value]

### OVERALL RECOMMENDATIONS
[Summary of findings and general recommendations for improvement]

## Instructions
- Be thorough but concise in your analysis
- Focus on educational appropriateness and student engagement
- Provide constructive feedback without making direct corrections
- Identify areas that need improvement while maintaining objectivity
"""


In [132]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [133]:
TEMPLATE_fix_json = """
You will receive a string that is meant to be JSON but may contain syntax errors like:
- Missing commas, quotes, or brackets
- Trailing commas or unbalanced braces
- Wrongly escaped characters

Your task:
✅ Repair the string into valid JSON.  
✅ Return only **valid JSON output** with no surrounding text or formatting.  
✅ Do **not** wrap the JSON in markdown code fences (```), text like "Here is the JSON:", or any explanation.  
✅ Output only raw JSON.  

Here is the broken JSON string:
{quiz}
"""

In [134]:
fix_json_prompt = PromptTemplate(
    input_variables=["quiz"],
    template= TEMPLATE_fix_json 
)

fix_json_chain = LLMChain(
    llm=llm_json_fixer,
    prompt=fix_json_prompt,
    output_key="fixed_quiz",
    verbose=True
)

In [135]:
fix_json_chain = LLMChain(
    llm=llm_json_fixer,
    prompt=fix_json_prompt,
    output_key="fixed_quiz",
    verbose=True
)

In [103]:
# review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [136]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain, fix_json_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review", "fixed_quiz"],
    verbose=False
)

In [137]:
file_path = "D:\MCQ_Generator\data.txt"

In [138]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [139]:
NUMBER= 5
SUBJECT="computer vision"
TONE="simple"

In [140]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

SYSTEM: You are a JSON-only MCQ generator. You must output ONLY valid JSON.

INPUT TEXT:
Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] "Understanding" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. This image understanding can be seen as the disentangling of symbolic information from image data using models constructed with the aid of geometry, physics, statistics, and learning theory.

The scientific discipline of computer vision is concerned with the theory behind artificial systems that extract information from images. Image data can take many forms, such as video sequences, views from multiple cam

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

# Quiz Review and Analysis Instructions

## Input Quiz
{
"quiz_info": {
"title": "Computer Vision Quiz",
"subject": "Computer Vision",
"difficulty": "easy",
"total_questions": 5
},
"questions": [
{
"id": 1,
"question": "What is the main topic discussed in the text?",
"options": {
"A": "Acquiring, processing, analyzing, and understanding digital images",
"B": "Machine learning algorithms",
"C": "Image compression techniques",
"D": "Video editing software"
},
"correct_answer": "A",
"explanation": "The text discusses the main topic of computer vision, which is acquiring, processing, analyzing, and understanding digital images."
},
{
"id": 2,
"question": "What is the scientific discipline of computer vision concerned with?",
"options": {
"A": "Image compression",
"B": "Theory behind artificial systems that extract information from images",
"C": "Image recognition",
"D": "Video editing"
},
"correct_answer": "B",
"explanation": "The scientific dis

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You will receive a string that is meant to be JSON but may contain syntax errors like:
- Missing commas, quotes, or brackets
- Trailing commas or unbalanced braces
- Wrongly escaped characters

Your task:
✅ Repair the string into valid JSON.  
✅ Return only **valid JSON output** with no surrounding text or formatting.  
✅ Do **not** wrap the JSON in markdown code fences (```), text like "Here is the JSON:", or any explanation.  
✅ Output only raw JSON.  

Here is the broken JSON string:
{
"quiz_info": {
"title": "Computer Vision Quiz",
"subject": "Computer Vision",
"difficulty": "easy",
"total_questions": 5
},
"questions": [
{
"id": 1,
"question": "What is the main topic discussed in the text?",
"options": {
"A": "Acquiring, processing, analyzing, and understanding digital images",
"B": "Machine learning algorithms",
"C": "Image compression techniques",
"D": "Video editing software"
},
"correct_answer": "A",
"explanation": "The text discusse

In [109]:
import pandas as pd
from datetime import datetime

In [110]:
# Check what variables you have
print("Available variables:")
print([var for var in dir() if not var.startswith('_')])

# If you have a response, check its name
# Common names might be: response, result, mcqs, mcq_response, etc.

Available variables:
['ChatGroq', 'In', 'LLMChain', 'NUMBER', 'Out', 'PromptTemplate', 'PyPDF2', 'RESPONSE_JSON', 'SUBJECT', 'SequentialChain', 'TEMPLATE', 'TEMPLATE2', 'TEMPLATE_extract_analysis', 'TEMPLATE_extract_quiz', 'TEMPLATE_fix_json', 'TEXT', 'TONE', 'datetime', 'exit', 'extract_analysis_chain', 'extract_analysis_prompt', 'extract_quiz_chain', 'extract_quiz_prompt', 'file', 'file_path', 'fix_json_chain', 'fix_json_prompt', 'generate_evaluate_chain', 'get_ipython', 'json', 'json_str', 'key', 'llm', 'llm_json_fixer', 'load_dotenv', 'open', 'os', 'pd', 'quit', 'quiz_chain', 'quiz_evaluation_prompt', 'quiz_generation_prompt', 'response', 'review_chain', 'save_mcqs_to_csv', 'traceback']


In [111]:
# Check the type and structure of your response
print(f"Response type: {type(response)}")
print(f"Response content: {response}")

# If it's a dictionary, check its keys
if isinstance(response, dict):
    print(f"Dictionary keys: {list(response.keys())}")
    # Common keys might be: 'content', 'text', 'response', 'result', etc.

Response type: <class 'dict'>
Response content: {'text': 'Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] "Understanding" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. This image understanding can be seen as the disentangling of symbolic information from image data using models constructed with the aid of geometry, physics, statistics, and learning theory.\n\nThe scientific discipline of computer vision is concerned with the theory behind artificial systems that extract information from images. Image data can take many forms, such as video sequences, views from multiple cameras, multi-dimensional data from a 3D scanner, 3D poin

In [141]:
print(response["review"])

**COMPLEXITY ANALYSIS**
The questions in this quiz are generally well-suited for computer vision students, with a moderate level of cognitive complexity. The questions require students to apply their knowledge of computer vision concepts, but do not demand overly complex problem-solving or abstract thinking.

**QUALITY ASSESSMENT**
The quiz questions are well-written, with clear and concise language. The options provided are plausible and relevant to the topic, making it easy for students to understand the questions. The explanations provided for each question are also clear and helpful. However, some questions may benefit from additional context or examples to further clarify the concepts.

**DIFFICULTY EVALUATION**
The difficulty level of the questions is generally appropriate for computer vision students. The questions are challenging enough to require students to think critically, but not so difficult that they become overwhelming. However, some questions may be more challenging th

In [113]:
# Extract the quiz content from your response
quiz_content = response['quiz']
print("🔍 YOUR MCQ QUIZ:")
print("=" * 80)
print(quiz_content)
print("=" * 80)

🔍 YOUR MCQ QUIZ:
{
"quiz_info": {
"title": "Computer Vision Quiz",
"subject": "Computer Vision",
"difficulty": "easy",
"total_questions": 3
},
"questions": [
{
"id": 1,
"question": "What is the main topic discussed in the text?",
"options": {
"A": "Acquiring, processing, analyzing, and understanding digital images",
"B": "Scene reconstruction and object detection",
"C": "Image restoration and 3D scene modeling",
"D": "Event detection and activity recognition"
},
"correct_answer": "A",
"explanation": "The text discusses the main topic of computer vision, which is acquiring, processing, analyzing, and understanding digital images."
},
{
"id": 2,
"question": "Which of the following is a subdiscipline of computer vision?",
"options": {
"A": "Image compression",
"B": "Object recognition",
"C": "Scene reconstruction",
"D": "Data mining"
},
"correct_answer": "B",
"explanation": "Object recognition is a subdiscipline of computer vision."
},
{
"id": 3,
"question": "What is the goal of computer 

In [114]:
# import pandas as pd
# from datetime import datetime

# # Function to save MCQs to CSV
# def save_mcqs_to_csv(quiz_data, filename=None):
#     """Save MCQs to CSV format"""
#     if not quiz_data or 'questions' not in quiz_data:
#         print("No quiz data to save")
#         return
    
#     # Create rows for CSV
#     rows = []
#     for question in quiz_data['questions']:
#         row = {
#             'Question_Number': question.get('id', ''),
#             'Question': question.get('question', ''),
#             'Option_A': question.get('options', {}).get('A', ''),
#             'Option_B': question.get('options', {}).get('B', ''),
#             'Option_C': question.get('options', {}).get('C', ''),
#             'Option_D': question.get('options', {}).get('D', ''),
#             'Correct_Answer': question.get('correct_answer', ''),
#             'Explanation': question.get('explanation', '')
#         }
#         rows.append(row)
    
#     # Create DataFrame
#     df = pd.DataFrame(rows)
    
#     # Generate filename if not provided
#     if filename is None:
#         timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#         subject = quiz_data.get('quiz_info', {}).get('subject', 'quiz').replace(' ', '_').lower()
#         filename = f"{subject}_mcqs_{timestamp}.csv"
    
#     # Save to CSV
#     df.to_csv(filename, index=False, encoding='utf-8')
#     print(f"✅ MCQs saved to CSV: {filename}")
#     print(f"📊 Total questions saved: {len(rows)}")
    
#     return filename

# # Save your current MCQs to CSV
# if quiz_data:
#     csv_filename = save_mcqs_to_csv(quiz_data)
    
#     # Display preview of the CSV
#     print(f"\n�� CSV Preview:")
#     print("=" * 80)
#     df_preview = pd.read_csv(csv_filename)
#     print(df_preview.head())
    
#     print(f"\n📁 File saved as: {csv_filename}")
# else:
#     print("No quiz data available to save")

In [115]:
print(response["fixed_quiz"])

{
"quiz_info": {
"title": "Computer Vision Quiz",
"subject": "Computer Vision",
"difficulty": "easy",
"total_questions": 3
},
"questions": [
{
"id": 1,
"question": "What is the main topic discussed in the text?",
"options": {
"A": "Acquiring, processing, analyzing, and understanding digital images",
"B": "Scene reconstruction and object detection",
"C": "Image restoration and 3D scene modeling",
"D": "Event detection and activity recognition"
},
"correct_answer": "A",
"explanation": "The text discusses the main topic of computer vision, which is acquiring, processing, analyzing, and understanding digital images."
},
{
"id": 2,
"question": "Which of the following is a subdiscipline of computer vision?",
"options": {
"A": "Image compression",
"B": "Object recognition",
"C": "Scene reconstruction",
"D": "Data mining"
},
"correct_answer": "B",
"explanation": "Object recognition is a subdiscipline of computer vision."
},
{
"id": 3,
"question": "What is the goal of computer vision?",
"option

In [116]:
# from groq import Groq
# GROQ_API_KEY = key

# # Initialize Groq client
# client = Groq(api_key=GROQ_API_KEY)

# # Define the prompt template
# fixing_string = PromptTemplate(
#     input_variables=["quiz"],
#     template="""
# You will receive a string that is meant to be JSON but may contain syntax errors like:
# - Missing commas, quotes, or brackets
# - Trailing commas or unbalanced braces
# - Wrongly escaped characters

# Your task:
# ✅ Repair the string into valid JSON.  
# ✅ Return only **valid JSON output** with no surrounding text or formatting.  
# ✅ Do **not** wrap the JSON in markdown code fences (```), text like "Here is the JSON:", or any explanation.  
# ✅ Output only raw JSON.  

# Here is the broken JSON string:
# {quiz}
# """
# )


In [117]:
# def fix_json(broken_json: str) -> str:
#     """
#     Takes a broken JSON string, sends it to LLaMA 3 on Groq API,
#     and returns the fixed, pretty-printed JSON string.
#     """
#     # Format the prompt with the malformed JSON
#     prompt = fixing_string.format(quiz=broken_json)

#     # Call Groq API
#     response = client.chat.completions.create(
#         model="llama3-70b-8192",  # Or "mixtral-8x7b-32768" if you prefer faster
#         messages=[
#             {"role": "system", "content": "You are a JSON fixing assistant."},
#             {"role": "user", "content": prompt}
#         ],
#         max_tokens=4096,
#         temperature=0  # Deterministic output
#     )

#     # Extract the fixed JSON
#     fixed_json = response.choices[0].message.content.strip()
#     return fixed_json

In [142]:
import json
import pandas as pd
from datetime import datetime

def save_mcqs_to_csv(json_string, filename=None):
    """
    Save quiz data (quiz_info + questions) from a JSON string to CSV files.

    Parameters:
    - json_string (str): The quiz data as a JSON string.
    - filename (str, optional): The base filename for CSVs. If not provided, generates one.

    Returns:
    - tuple: Filenames of the saved CSV files (quiz_info_file, questions_file).
    """
    try:
        # Parse JSON string
        quiz_data = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"❌ Failed to parse JSON string: {e}")
        return

    if not quiz_data or 'questions' not in quiz_data:
        print("❌ No quiz data to save")
        return

    # Create DataFrames
    quiz_info_df = pd.DataFrame([quiz_data.get("quiz_info", {})])
    questions_df = pd.DataFrame(quiz_data["questions"])

    # Generate base filename if not provided
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        subject = quiz_data.get('quiz_info', {}).get('subject', 'quiz').replace(' ', '_').lower()
        filename_base = f"{subject}_mcqs_{timestamp}"
    else:
        filename_base = filename.replace('.csv', '')  # Remove .csv if passed

    # Create filenames for quiz_info and questions
    quiz_info_file = f"{filename_base}_info.csv"
    questions_file = f"{filename_base}_questions.csv"

    # Save to CSV
    quiz_info_df.to_csv(quiz_info_file, index=False, encoding="utf-8")
    questions_df.to_csv(questions_file, index=False, encoding="utf-8")

    print(f"✅ Quiz Info saved to: {quiz_info_file}")
    print(f"✅ Questions saved to: {questions_file}")
    print(f"📊 Total questions saved: {len(questions_df)}")

    return quiz_info_file, questions_file


In [143]:
# Example JSON string (from LLM)
json_str = response["fixed_quiz"]  # Directly from your chain

# Save to CSV
save_mcqs_to_csv(json_str)


✅ Quiz Info saved to: computer_vision_mcqs_20250706_040808_info.csv
✅ Questions saved to: computer_vision_mcqs_20250706_040808_questions.csv
📊 Total questions saved: 5


('computer_vision_mcqs_20250706_040808_info.csv',
 'computer_vision_mcqs_20250706_040808_questions.csv')

In [120]:
# review = response["review"]
# print(review)